### Play around with trajectory analysis
Daniel O'Donohue from MapScaping recommended me movingpandas and mobilitydb, so I'll try out the first one here

based on Geospatial Programming's YT Video https://www.youtube.com/watch?v=4S3f_fqInZY

### struggeled with install of GeoViews and cartopy (pip install didn't work) until I discovered, that they explicitly say that it's not recommended to install it via pip
-> trying out the Anaconda approach now.  <br> 
-> didn't work neither. Followed the steps described on the moving pandas GitHub for Anaconda, but got 500: Internal Server Error when trying to open the .ipynb file. 
Stackoverflow indicated that the problem might be that the package nbconvert is at version 5.5.0 (newest is 7.4.0), but updating in Anaconda doesn’t work <br> 
-> let it be for now. No movingpandas, but start simple with GeoPandas (see other .ipynb) <br> 
-> update Mid-June: plan to try it out with venv in PyCharm after getting some experience with virtual environments in R

In [ ]:
# !pip install GeoViews
# !pip install cartopy

fails to build cartopy...

In [ ]:
# !pip install movingpandas
import GeoViews
import cartopy

In [ ]:
import pandas as pd
import movingpandas as mpd

# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("JaredCombs_Ankle_Monitor.csv")

In [ ]:
df.head(3)

In [ ]:
df.dtypes

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
df.shape

In [ ]:
df['datetime'].min(), df['datetime'].max()
# we see: the observations stem from 5 different days in November 2016

In [ ]:
df = df[df['datetime'] < '2016-11-08'] # filter only the first day

In [ ]:
traj = mpd.Trajectory(df, "SCombs", t="datetime", x="location_x", y="location_y", crs=4326)
# create trajectory, give it some ID (doesn't matter that much here as it's only one person / single-entity)
# define projection

In [ ]:
traj.df

In [ ]:
traj.hvplot(c='speed', line_width=4, colorbar=True)

In [ ]:
from datetime import timedelta

In [ ]:
detector = mpd.TrajectoryStopDetector(traj)

In [ ]:
stops = detector.get_stop_points(min_duration=timedelta(seconds=600), max_diameter=50)

In [ ]:
stops

In [ ]:
stops = stops.set_crs(4326)

In [ ]:
import hvplot.pandas

In [ ]:
stops.hvplot(geo=True, tiles=True, hover_cols="all", frame_height=300, frame_width=300) + \
traj.hvplot(geo=True, tiles=True, hover_cols="all", frame_height=300, frame_width=300)

In [ ]:
traj_col = mpd.StopSplitter(traj).split(max_diameter=50, min_duration=timedelta(seconds=600))

In [ ]:
traj_col.hvplot(line_width=3)

In [ ]:
import geopandas as gpd

In [ ]:
crimes = gpd.read_file("crime_reports.shp")

In [ ]:
crimes

In [ ]:
from pyproj import CRS

In [ ]:
traj = traj.to_crs(CRS(3857))

In [ ]:
crimes = crimes.to_crs(3857)

In [ ]:
crimes['dt'] = pd.to_datetime(crimes['report_dat'])

In [ ]:
crimes = crimes[crimes['dt'] < '2016-11-08']

In [ ]:
crimes

In [ ]:
crimes['sus_loc'] = crimes['dt'].apply(traj.get_position_at)

In [ ]:
crimes['distance'] = crimes.apply(lambda row: row['geometry'].distance(row['sus_loc']), axis=1)

In [ ]:
crimes[crimes['distance'] < 200]

In [ ]:
stops.hvplot(geo=True, tiles=True, frame_height=400, frame_width=400) + \
crimes[crimes['distance'] < 200].hvplot(tiles=True, frame_height=400, frame_width=400)